# Multi-Head Diversity: Randomized Stressors & Robustness Visualization

**Randomized stressors** (not worst-case PGD/FGSM or certified): Finance-valid attacks A1–A4 as distributional stress tests.

**Model**: Multi-Head Diversity only.

**Metrics**: R²-based and RMSE-based robustness, with rich visualizations common in adversarial training and certification.

In [ ]:
USE_SMALL_DATA = True
N_SAMPLES = 5000
N_EPOCHS = 30
N_STRESS_RUNS = 5  # Number of random runs per (attack, epsilon) for robustness CI

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print('Google Drive mounted.')
except Exception:
    pass

In [ ]:
import sys, os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

def _find_repo_root():
    cwd = Path.cwd().resolve()
    for p in [Path('/content/drive/MyDrive/multihead-attention-robustness'), Path('/content/drive/My Drive/multihead-attention-robustness'), Path('/content/repo_run')]:
        if p.exists() and (p / 'src').exists():
            return p
    drive_root = Path('/content/drive')
    if drive_root.exists():
        for base in [drive_root / 'MyDrive', drive_root / 'My Drive', drive_root]:
            if base.exists():
                for sub in base.iterdir():
                    if sub.is_dir() and 'multihead-attention' in sub.name.lower() and (sub / 'src').exists():
                        return sub
    p = cwd
    for _ in range(10):
        if (p / 'src').exists():
            return p
        if p.parent == p:
            break
        p = p.parent
    return cwd.parent if cwd.name == 'notebooks' else cwd

repo_root = _find_repo_root()
if not (repo_root / 'src').exists():
    raise FileNotFoundError(f"Repo root not found. Run Drive mount first.")
sys.path.insert(0, str(repo_root))
os.chdir(repo_root)
from src.models.feature_token_transformer import FeatureTokenTransformer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}, Repo: {repo_root}')

## 1. Load Data

In [ ]:
data_path = repo_root / 'data' / 'cross_sectional' / 'master_table.csv'
if not data_path.exists():
    data_path = repo_root / 'data' / 'master_table.csv'
df = pd.read_csv(data_path)
if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date')

class CrossSectionalDataSplitter:
    def __init__(self, train_start='2005-01-01', train_end='2017-12-31', val_start='2018-01-01', val_end='2019-12-31'):
        self.train_start, self.train_end = train_start, train_end
        self.val_start, self.val_end = val_start, val_end
    def split(self, master_table):
        master_table = master_table.copy()
        master_table.index = pd.to_datetime(master_table.index)
        return {'train': master_table.loc[self.train_start:self.train_end], 'val': master_table.loc[self.val_start:self.val_end]}
    def prepare_features_labels(self, data):
        if data.empty:
            return pd.DataFrame(), pd.Series()
        numeric_data = data.select_dtypes(include=[np.number])
        if numeric_data.empty:
            return pd.DataFrame(), pd.Series()
        exclude_cols = ['mktcap', 'market_cap', 'date', 'year', 'month', 'ticker', 'permno', 'gvkey']
        target_cols = ['return', 'returns', 'ret', 'target', 'y', 'next_return', 'forward_return', 'ret_1', 'ret_1m', 'ret_12m', 'future_return', 'returns_1d']
        target_col = None
        for tc in target_cols:
            for col in numeric_data.columns:
                if tc.lower() in col.lower() and col.lower() not in [ec.lower() for ec in exclude_cols]:
                    target_col = col
                    break
            if target_col:
                break
        if target_col is None:
            potential = [c for c in numeric_data.columns if c.lower() not in [ec.lower() for ec in exclude_cols]]
            target_col = potential[-2] if len(potential) > 1 else (potential[-1] if potential else numeric_data.columns[-1])
        feature_cols = [c for c in numeric_data.columns if c != target_col and c.lower() not in [ec.lower() for ec in exclude_cols]]
        if not feature_cols:
            feature_cols = [c for c in numeric_data.columns if c != target_col]
        if not feature_cols:
            feature_cols = numeric_data.columns[:-1].tolist()
            target_col = numeric_data.columns[-1]
        return numeric_data[feature_cols], numeric_data[target_col]

splitter = CrossSectionalDataSplitter()
data_splits = splitter.split(df)
train_df, val_df = data_splits['train'], data_splits['val']
X_train_df, y_train = splitter.prepare_features_labels(train_df)
X_val_df, y_val = splitter.prepare_features_labels(val_df)
X_train = X_train_df.fillna(0).values.astype(np.float32)
y_train = y_train.fillna(0).values.astype(np.float32)
X_val = X_val_df.fillna(0).values.astype(np.float32)
y_val = y_val.fillna(0).values.astype(np.float32)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
if USE_SMALL_DATA and N_SAMPLES < len(X_train):
    idx = np.random.RandomState(RANDOM_SEED).choice(len(X_train), N_SAMPLES, replace=False)
    X_train_scaled, y_train = X_train_scaled[idx], y_train[idx]

sigma_train = np.std(X_train_scaled, axis=0) + 1e-8
val_dates = X_val_df.index if hasattr(X_val_df, 'index') else None
print(f'Data: train {X_train_scaled.shape[0]}, val {X_val_scaled.shape[0]}, features {X_train_scaled.shape[1]}')

## 2. Randomized Stressors (A1–A4)

In [ ]:
def apply_a1(X, epsilon, sigma):
    """A1: Measurement Error - η_i ~ N(0, ε²I) per sample."""
    eta = np.random.normal(0, epsilon, X.shape)
    return X + eta * sigma

def apply_a2(X, epsilon):
    """A2: Missingness - zero fraction p = min(ε/10, 0.8) of features."""
    p = min(epsilon / 10.0, 0.8)
    X_adv = X.copy()
    n_samples, n_features = X.shape
    n_missing = max(1, int(n_features * p))
    for i in range(n_samples):
        idx = np.random.choice(n_features, n_missing, replace=False)
        X_adv[i, idx] = 0.0
    return X_adv

def apply_a3(X, epsilon, sigma):
    """A3: Rank Manipulation - δ_i,j ~ N(0, ε² σ_j²)."""
    delta = np.random.normal(0, epsilon, X.shape) * sigma
    return X + delta

def apply_a4(X, epsilon, sigma):
    """A4: Regime Shift - shared η_t across samples (simplified: per batch)."""
    eta = np.random.normal(0, 1, X.shape[1])
    return X + epsilon * sigma * eta

ATTACKS = {'A1': 'Measurement Error', 'A2': 'Missingness', 'A3': 'Rank Manipulation', 'A4': 'Regime Shift'}
EPSILONS = [0.25, 0.5, 1.0]

## 3. Train Multi-Head Diversity

In [ ]:
def train_mhd(model, X_train, y_train, X_val, y_val, config, device):
    model = model.to(device)
    criterion = nn.MSELoss()
    opt = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
    X_t = torch.FloatTensor(X_train).to(device)
    y_t = torch.FloatTensor(y_train).to(device)
    X_v = torch.FloatTensor(X_val).to(device)
    batch_size = config['batch_size']
    for epoch in range(config['epochs']):
        model.train()
        for i in range(0, len(X_t), batch_size):
            bx, by = X_t[i:i+batch_size], y_t[i:i+batch_size]
            opt.zero_grad()
            pred, attn = model(bx)
            loss = criterion(pred.squeeze(), by)
            if model.use_head_diversity and attn:
                loss = loss + model.compute_diversity_loss([attn[f'layer_{j}'] for j in range(len(attn))])
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
        if (epoch + 1) % 20 == 0:
            model.eval()
            with torch.no_grad():
                p, _ = model(X_v)
                vloss = criterion(p.squeeze(), torch.FloatTensor(y_val).to(device))
            print(f'  Epoch {epoch+1}: val_loss={vloss.item():.6f}')
    
    model.eval()
    with torch.no_grad():
        pred, _ = model(X_v)
        pred = pred.squeeze().cpu().numpy()
    return model, pred

tr_cfg = {'d_model': 72, 'num_layers': 2, 'd_ff': 512, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 32, 'epochs': N_EPOCHS if USE_SMALL_DATA else 100}
model = FeatureTokenTransformer(num_features=X_train_scaled.shape[1], d_model=tr_cfg['d_model'], num_heads=8, num_layers=tr_cfg['num_layers'], d_ff=tr_cfg['d_ff'], dropout=tr_cfg['dropout'], use_head_diversity=True, diversity_weight=0.01)
model, pred_clean = train_mhd(model, X_train_scaled, y_train, X_val_scaled, y_val, tr_cfg, device)

clean_rmse = np.sqrt(mean_squared_error(y_val, pred_clean))
clean_r2 = r2_score(y_val, pred_clean)
print(f'Clean: RMSE={clean_rmse:.6f}, R²={clean_r2:.6f}')

## 4. Evaluate Under Stressors

In [ ]:
def evaluate_under_stress(model, X_val, y_val, sigma, attack_name, epsilon, n_runs, device, clean_rmse, clean_r2):
    model.eval()
    rmses, r2s, rob_rmse, rob_r2 = [], [], [], []
    for _ in range(n_runs):
        if attack_name == 'A1':
            X_adv = apply_a1(X_val, epsilon, sigma)
        elif attack_name == 'A2':
            X_adv = apply_a2(X_val, epsilon)
        elif attack_name == 'A3':
            X_adv = apply_a3(X_val, epsilon, sigma)
        elif attack_name == 'A4':
            X_adv = apply_a4(X_val, epsilon, sigma)
        else:
            X_adv = X_val.copy()
        
        with torch.no_grad():
            X_t = torch.FloatTensor(X_adv).to(device)
            pred, _ = model(X_t)
            pred = pred.squeeze().cpu().numpy()
        
        rmse = np.sqrt(mean_squared_error(y_val, pred))
        r2 = r2_score(y_val, pred)
        rmses.append(rmse)
        r2s.append(r2)
        delta_rmse = rmse - clean_rmse
        rob_rmse.append(min(1.0, 1.0 - delta_rmse / clean_rmse) if clean_rmse > 0 else 1.0)
        rob_r2.append(min(1.0, r2 / clean_r2) if clean_r2 > 0 else 1.0)
    
    return {'rmse_mean': np.mean(rmses), 'rmse_std': np.std(rmses), 'r2_mean': np.mean(r2s), 'r2_std': np.std(r2s),
            'rob_rmse_mean': np.mean(rob_rmse), 'rob_rmse_std': np.std(rob_rmse),
            'rob_r2_mean': np.mean(rob_r2), 'rob_r2_std': np.std(rob_r2),
            'delta_rmse': np.mean(rmses) - clean_rmse}

results = []
for attack in ATTACKS:
    for eps in EPSILONS:
        r = evaluate_under_stress(model, X_val_scaled, y_val, sigma_train, attack, eps, N_STRESS_RUNS, device, clean_rmse, clean_r2)
        results.append({'attack': attack, 'epsilon': eps, **r})

results_df = pd.DataFrame(results)
print(results_df.to_string())

## 5. Visualizations

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. RMSE-based robustness vs epsilon
ax = axes[0, 0]
for attack in ATTACKS:
    d = results_df[results_df['attack'] == attack]
    ax.errorbar(d['epsilon'], d['rob_rmse_mean'], yerr=d['rob_rmse_std'], fmt='o-', label=attack, capsize=3)
ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('ε (perturbation budget)')
ax.set_ylabel('Robustness (RMSE-based)')
ax.set_title('RMSE-based Robustness: R = min(1, 1 - ΔRMSE/RMSE_clean)')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. R² degradation vs epsilon
ax = axes[0, 1]
for attack in ATTACKS:
    d = results_df[results_df['attack'] == attack]
    ax.errorbar(d['epsilon'], d['r2_mean'], yerr=d['r2_std'], fmt='s-', label=attack, capsize=3)
ax.axhline(clean_r2, color='gray', linestyle='--', alpha=0.5, label='Clean')
ax.set_xlabel('ε')
ax.set_ylabel('R² under stress')
ax.set_title('R² Degradation Under Randomized Stressors')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Heatmap: RMSE-based robustness
ax = axes[1, 0]
pivot = results_df.pivot(index='attack', columns='epsilon', values='rob_rmse_mean')
sns.heatmap(pivot, annot=True, fmt='.3f', cmap='RdYlGn', vmin=0.5, vmax=1.0, ax=ax)
ax.set_title('Robustness Heatmap (RMSE-based)')

# 4. ΔRMSE by attack and epsilon
ax = axes[1, 1]
x = np.arange(len(ATTACKS))
width = 0.25
for i, eps in enumerate(EPSILONS):
    vals = [results_df[(results_df['attack']==a) & (results_df['epsilon']==eps)]['delta_rmse'].values[0] for a in ATTACKS]
    ax.bar(x + i*width, vals, width, label=f'ε={eps}')
ax.set_xticks(x + width)
ax.set_xticklabels(list(ATTACKS))
ax.set_ylabel('ΔRMSE')
ax.set_title('RMSE Degradation (ΔRMSE = RMSE_adv - RMSE_clean)')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.suptitle('Multi-Head Diversity: Randomized Stressors (A1–A4)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# R²-based robustness heatmap
if 'rob_r2_mean' in results_df.columns:
    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    pivot_r2 = results_df.pivot(index='attack', columns='epsilon', values='rob_r2_mean')
    sns.heatmap(pivot_r2, annot=True, fmt='.3f', cmap='RdYlGn', vmin=0.5, vmax=1.0, ax=ax)
    ax.set_title('R²-based Robustness: R²_rob = min(1, R²_adv/R²_clean)')
    plt.tight_layout()
    plt.show()

In [ ]:
# Clean vs Adversarial prediction scatter (one attack, one epsilon)
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

X_adv_a1 = apply_a1(X_val_scaled, 1.0, sigma_train)
with torch.no_grad():
    pred_adv = model(torch.FloatTensor(X_adv_a1).to(device))[0].squeeze().cpu().numpy()

axes[0].scatter(pred_clean, pred_adv, alpha=0.3, s=5)
axes[0].plot([pred_clean.min(), pred_clean.max()], [pred_clean.min(), pred_clean.max()], 'r--', label='y=x')
axes[0].set_xlabel('Clean prediction')
axes[0].set_ylabel('Adversarial prediction (A1, ε=1.0)')
axes[0].set_title('Clean vs Adversarial Predictions')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].scatter(y_val, pred_clean, alpha=0.3, s=5, label='Clean', c='blue')
axes[1].scatter(y_val, pred_adv, alpha=0.3, s=5, label='A1 ε=1.0', c='orange')
axes[1].set_xlabel('True return')
axes[1].set_ylabel('Prediction')
axes[1].set_title('True vs Predicted (Clean vs Stressed)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Time series of predictions under stress (if dates available)
if val_dates is not None and len(val_dates) == len(y_val):
    fig, ax = plt.subplots(figsize=(12, 4))
    dates = pd.to_datetime(val_dates)
    ax.plot(dates, y_val, 'k-', alpha=0.5, label='True', linewidth=0.5)
    ax.plot(dates, pred_clean, 'b-', alpha=0.7, label='Clean', linewidth=0.5)
    X_adv_ts = apply_a1(X_val_scaled, 1.0, sigma_train)
    with torch.no_grad():
        pred_adv_ts = model(torch.FloatTensor(X_adv_ts).to(device))[0].squeeze().cpu().numpy()
    ax.plot(dates, pred_adv_ts, 'orange', alpha=0.7, label='A1 ε=1.0', linewidth=0.5)
    ax.set_xlabel('Date')
    ax.set_ylabel('Return / Prediction')
    ax.set_title('Time Series: Predictions Under Randomized Stress (A1)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
print('='*60)
print('SUMMARY: Multi-Head Diversity (Randomized Stressors)')
print('='*60)
print(f'Clean: RMSE={clean_rmse:.6f}, R²={clean_r2:.6f}')
print('\nRobustness (RMSE-based) by attack (mean over ε):')
for a in ATTACKS:
    m = results_df[results_df['attack']==a]['rob_rmse_mean'].mean()
    print(f'  {a}: {m:.4f}')
if 'rob_r2_mean' in results_df.columns:
    print('\nRobustness (R²-based) by attack (mean over ε):')
    for a in ATTACKS:
        m = results_df[results_df['attack']==a]['rob_r2_mean'].mean()
        print(f'  {a}: {m:.4f}')
print('\nRobustness (RMSE-based) by ε (mean over attacks):')
for e in EPSILONS:
    m = results_df[results_df['epsilon']==e]['rob_rmse_mean'].mean()
    print(f'  ε={e}: {m:.4f}')